# Example notebook for UMAP with CML data

### Imports and settings

In [1]:
%matplotlib widget
import numpy as np
import umap
import xarray as xr
import numba
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# fix random seed for reproducibility
random_state = 1234
np.random.seed(random_state)
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Load data

In [3]:
fn='umap_15_link_subset.nc'

In [4]:
ds=xr.open_dataset(fn).load()

In [5]:
ds

<xarray.Dataset>
Dimensions:    (n: 32478, x: 60)
Dimensions without coordinates: n, x
Data variables:
    samples    (n, x) float64 65.0 64.0 65.0 64.0 64.0 ... 56.8 55.8 56.8 56.8
    radar      (n) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    rain_rate  (n) float64 0.687 2.562 2.312 0.4484 ... 0.0 0.07475 0.0 0.0
    sensor_id  (n) object 'SY2421_2_SY2402_6' ... 'HY1309_2_HY8929_2'
    timestamp  (n) datetime64[ns] 2017-10-22T13:51:00 ... 2017-10-14T08:51:00

In [6]:
samples = ds.samples.values
radar = ds.radar.values
rain_rate = ds.rain_rate.values
sensor_id = ds.sensor_id.values
timestamp = ds.timestamp.values
samples.shape

(32478, 60)

In [7]:
rain_rate_x=rain_rate.copy()
rain_rate_x[rain_rate_x<0.1]=0.0001

In [8]:
id_list_num=le.fit_transform(sensor_id)

### Some statistics

In [9]:
sns.distplot(rain_rate, kde=False)
plt.yscale('log')
plt.ylabel('count []')
plt.xlabel('rain rate [mmh$^{-1}$]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
print('There are ',len(np.unique(sensor_id)),' different CMLs in the data set.')
print('The dataset consists of ',int((max(timestamp)-min(timestamp))/ np.timedelta64(1,'D')), ' days of data.')
print('Thats ',int((max(timestamp)-min(timestamp))/ np.timedelta64(1,'h')), ' hours.')

There are  15  different CMLs in the data set.
The dataset consists of  425  days of data.
Thats  10220  hours.


### Imports and settings

In [11]:
%%time
k=4
trans = umap.UMAP(n_neighbors=k, # number of neighbours for manifold approximation
                  min_dist=0.1, # minimum distance of points
                  n_components=2, # dimension of low dimensional representation
                  metric='euclidean', # metric for distance between points in high dimensional space
                  random_state=random_state,
                  verbose=1
                 ).fit(samples) # the actual data we use for approximation

UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
     learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
     metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
     n_neighbors=4, negative_sample_rate=5, random_state=1234,
     repulsion_strength=1.0, set_op_mix_ratio=1.0, spread=1.0,
     target_metric='categorical', target_metric_kwds=None,
     target_n_neighbors=-1, target_weight=0.5, transform_queue_size=4.0,
     transform_seed=42, verbose=1)
Construct fuzzy simplicial set
Tue Feb 18 17:29:13 2020 Finding Nearest Neighbors
Tue Feb 18 17:29:13 2020 Building RP forest with 14 trees
Tue Feb 18 17:29:14 2020 NN descent for 15 iterations
	 0  /  15
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
Tue Feb 18 17:29:16 2020 Finished Nearest Neighbor Search
Tue Feb 18 17:29:17 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	comple

In [12]:
%%time
embedding2d = trans.transform(samples)
xs2 = embedding2d[:,0]
ys2 = embedding2d[:,1]

CPU times: user 18.7 ms, sys: 148 µs, total: 18.8 ms
Wall time: 17.5 ms


In [27]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2, ys2, c=radar, picker=5, s=1, cmap='bwr_r')

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples[dataind], label=sensor_id[dataind]+' | '+str(np.round(rain_rate[dataind],decimals=2))+' mmh$^{-1}$')
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
axs[0].set_title('wet is blue and dry is red')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2, ys2, c=rain_rate_x, picker=5, s=1, cmap='bwr_r', norm=mpl.colors.LogNorm())

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples[dataind], label=sensor_id[dataind]+' | '+str(np.round(rain_rate[dataind],decimals=2))+' mmh$^{-1}$')
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
plt.colorbar(col, ax=axs[0], label='rain rate [mmh$^{-1}$]')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2, ys2, c=id_list_num, picker=5, s=1, cmap=plt.cm.get_cmap('viridis', 15))

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples[dataind], label=sensor_id[dataind]+' '+str(np.round(rain_rate[dataind],decimals=2)))
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
plt.colorbar(col, ax=axs[0], label='CML_id')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Normalize samples and redo analysis

In [16]:
samples_norm = samples.copy()
samples_norm = [x-y for x,y in zip(samples_norm, np.min(samples_norm, axis=-1))]

In [17]:
%%time
k=4
trans_n = umap.UMAP(n_neighbors=k, # number of neighbours for manifold approximation
                  min_dist=0.1, # minimum distance of points
                  n_components=2, # dimension of low dimensional representation
                  metric='euclidean', # metric for distance between points in high dimensional space
                  random_state=random_state,
                  verbose=1
                 ).fit(samples_norm)

UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
     learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
     metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
     n_neighbors=4, negative_sample_rate=5, random_state=1234,
     repulsion_strength=1.0, set_op_mix_ratio=1.0, spread=1.0,
     target_metric='categorical', target_metric_kwds=None,
     target_n_neighbors=-1, target_weight=0.5, transform_queue_size=4.0,
     transform_seed=42, verbose=1)
Construct fuzzy simplicial set
Tue Feb 18 17:29:41 2020 Finding Nearest Neighbors
Tue Feb 18 17:29:41 2020 Building RP forest with 14 trees
Tue Feb 18 17:29:42 2020 NN descent for 15 iterations
	 0  /  15
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
Tue Feb 18 17:29:44 2020 Finished Nearest Neighbor Search
Tue Feb 18 17:29:44 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epo

In [18]:
%%time
embedding2dn = trans_n.transform(samples_norm)
xs2n = embedding2dn[:,0]
ys2n = embedding2dn[:,1]

CPU times: user 25.7 ms, sys: 329 µs, total: 26.1 ms
Wall time: 24.8 ms


In [19]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2n, ys2n, c=id_list_num, picker=5, s=1, cmap=plt.cm.get_cmap('viridis', 15))

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples_norm[dataind], label=sensor_id[dataind]+' '+str(np.round(rain_rate[dataind],decimals=2)))
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
plt.colorbar(col, ax=axs[0], label='CML_id')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2n, ys2n, c=radar, picker=5, s=1, cmap='bwr_r')

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples_norm[dataind], label=sensor_id[dataind]+' | '+str(np.round(rain_rate[dataind],decimals=2))+' mmh$^{-1}$')
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
axs[0].set_title('wet is blue and dry is red')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2n, ys2n, c=rain_rate_x, picker=5, s=1, cmap='bwr_r', norm=mpl.colors.LogNorm())

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples_norm[dataind], label=sensor_id[dataind]+' | '+str(np.round(rain_rate[dataind],decimals=2))+' mmh$^{-1}$')
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
plt.colorbar(col, ax=axs[0], label='rain rate [mmh$^{-1}$]')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# custom distance measures using numba

In [22]:
@numba.njit()
def std_dif(x,y):
    return np.abs(np.std(x)-np.std(y))

@numba.njit()
def path_cost_numba(x, y, accumulated_cost, distances):
    path = np.zeros((len(x)-1+len(y-1),2))
    path[0] = [len(x)-1, len(y)-1]
    cost = 0
    i = len(y)-1
    j = len(x)-1
    n=0
    while i>0 and j>0:
        n=n+1
        if i==0:
            j = j - 1
        elif j==0:
            i = i - 1
        else:
            if accumulated_cost[i-1, j] == min(accumulated_cost[i-1, j-1], accumulated_cost[i-1, j], accumulated_cost[i, j-1]):
                i = i - 1
            elif accumulated_cost[i, j-1] == min(accumulated_cost[i-1, j-1], accumulated_cost[i-1, j], accumulated_cost[i, j-1]):
                j = j-1
            else:
                i = i - 1
                j= j- 1
        path[n]=[j, i]
    path = path[0:n+2]
    for i in range(len(path)):
        a, b = path[i]
        cost = cost + distances[int(b), int(a)]
        
    return path, cost

@numba.njit()
def comp_acc_cost(x, y, distances):
    accumulated_cost = np.zeros((len(y), len(x)))
    accumulated_cost[0,0] = distances[0,0]
    for i in range(1, len(x)):
        accumulated_cost[0,i] = distances[0,i] + accumulated_cost[0, i-1]    
    for i in range(1, len(y)):
        accumulated_cost[i,0] = distances[i, 0] + accumulated_cost[i-1, 0]  
    for i in range(1, len(y)):
        for j in range(1, len(x)):
            accumulated_cost[i, j] = min(accumulated_cost[i-1, j-1], accumulated_cost[i-1, j], accumulated_cost[i, j-1]) + distances[i, j]
            
    return accumulated_cost

@numba.njit()
def dtw_cost(x, y):
    distances = np.zeros((len(y), len(x)))
    for i in range(len(y)):
        for j in range(len(x)):
            distances[i,j] = (x[j]-y[i])**2
    accumulated_cost = comp_acc_cost(x,y,distances)
    cost = path_cost_numba(x, y, accumulated_cost, distances)[1]
    
    return cost

In [23]:
%%time
k=15
trans_c = umap.UMAP(n_neighbors=k, # number of neighbours for manifold approximation
                      min_dist=0.1, # 
                      n_components=2, # dimension of low dimensional representation
                      metric=std_dif, # metric for distance between points in high dimensional space
                        random_state=random_state
                     ).fit(samples_norm)

CPU times: user 3min 57s, sys: 2min 32s, total: 6min 29s
Wall time: 1min 21s


In [24]:
%%time
embedding2d_c = trans_c.transform(samples_norm)
xs2_c = embedding2d_c[:,0]
ys2_c = embedding2d_c[:,1]

CPU times: user 17.9 ms, sys: 3 ms, total: 20.9 ms
Wall time: 20 ms


In [25]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2_c, ys2_c, c=radar, picker=5, s=1, cmap='bwr_r')

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples_norm[dataind], label=sensor_id[dataind]+' | '+str(np.round(rain_rate[dataind],decimals=2))+' mmh$^{-1}$')
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
axs[0].set_title('wet is blue and dry is red')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
# axs[0].set_title('click on point to plot time series')
  # 5 points tolerance
col = axs[0].scatter(xs2_c, ys2_c, c=rain_rate_x, picker=5, s=1, cmap='bwr_r', norm=mpl.colors.LogNorm())

def onpick(event):

    if event.artist!=col: return True

    N = len(event.ind)
    if not N: return True

    axs[1].clear()
    for subplotnum, dataind in enumerate(event.ind[:3]):

        axs[1].plot(np.arange(0,60), samples_norm[dataind], label=sensor_id[dataind]+' | '+str(np.round(rain_rate[dataind],decimals=2))+' mmh$^{-1}$')
        axs[1].set_ylabel('TRSL [dB]')
        axs[1].set_xlabel('time [minutes]')
        axs[1].set_xticks(np.arange(0,61,10))
        axs[1].legend()
        
    return True

fig.canvas.mpl_connect('pick_event', onpick)
plt.colorbar(col, ax=axs[0], label='rain rate [mmh$^{-1}$]')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …